In [22]:
from pandas import read_csv, to_datetime, DataFrame, concat
import matplotlib.pyplot as plt
from numpy import sqrt, exp, log,zeros, nan
from scipy.stats import probplot
from pingouin import multivariate_normality 

from statsmodels.tsa.vector_ar.var_model import VAR,VARResults
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,root_mean_squared_error

## Les fonctions qui seront utilisées 

In [2]:
def r2_ajuste(r2,n,p):

    return  1 - (  ((1 - r2) * (n - 1)) / (n - p - 1)  )

In [3]:
def invert_transformation(df_original, df_forecast, transfo):
    """
    
    Revert back the differencing to get the forecast to original scale.
    
    """
    df_train = df_original.copy()
    df_fc = df_forecast.copy()
    df_fcout = df_forecast.copy()
    columns = df_forecast.columns
    
    for col in columns:
        # Retirer le logarithme si nécessaire
        if transfo.loc[ 'loga'] == 1:
            df_fcout[col] = exp(df_fcout[col])
        
        # Récupérer l'ordre de différenciation
        diff_order = transfo.loc[ 'ordreP']
        
        # Inverser la différenciation uniquement si l'ordre est 3
        if diff_order > 0 :
            for i in range(diff_order):
                df_fcout[col].iloc[i] = df_train[col].iloc[i]
            for j in range(diff_order, len(df_train[col])):
                df_fcout.loc[j,col] = df_fcout.loc[j - diff_order,col] + df_fc.loc[j,col]
            
        elif diff_order == 0:
            continue
        else:
            print(f"Vous n'avez pas fourni un bon ordre de différentiation. il est de {diff_order}")

    return df_fcout

## import des dataframes d'informations

In [ ]:
dfOrig = read_csv("Product families over time.csv")
dfOrig = dfOrig.set_index('Product Family').T
dfOrig.head(4)

In [ ]:
from statsmodels.tsa.stattools import adfuller
df2 = dfOrig.copy()

# Liste des familles de séries temporelles

def determine_petit_truc(series):
    min_positive_value = series[series > 0].min()
    val = 0.00001
    return min_positive_value * val

# Fonction pour tester la stationnarité avec le test ADF
def test_stationarity(series):
    result = adfuller(series.dropna())
    return result[1]  # Retourne la p-value

# Fonction pour appliquer la transformation logarithmique
def apply_log(series, p):
    return log(series + p)

# Initialiser un DataFrame pour enregistrer les résultats
results_df = DataFrame(columns=["family", "ordreP", "loga"])
fams = df2.columns

results_list = []
stat_data = dict()
# Traiter chaque famille de séries temporelles
for family in fams:
    #print(f"\nTraitement de la famille: {family}")  
    series = df2[family].dropna()
    p_val = test_stationarity(series)
    ordreP = 0
    loga = 0
    petit_truc = determine_petit_truc(df2[family])
    if p_val < 0.05:
        stat_data[family] = series
    # Tester la stationnarité de la série originale  
    if p_val > 0.05:
        # Appliquer des différentiations successives
        for p in range(1, 6):
            diff_series = series.diff(periods=p).dropna()
            p_val = test_stationarity(diff_series)
            if p_val <= 0.05:
                ordreP = p
                stat_data[family] = diff_series
                break    
        # Si toujours non stationnaire après différentiation
        if p_val > 0.05:
            # Appliquer une transformation logarithmique
            log_series = apply_log(series, p=petit_truc)
            loga = 1        
            # Tester la stationnarité de la série log-transformée
            p_val = test_stationarity(log_series)
            if p_val > 0.05:
                # Appliquer des différentiations successives sur la série log-transformée
                for p in range(1, 6):
                    diff_log_series = log_series.diff(periods=p).dropna()
                    p_val = test_stationarity(diff_log_series)
                    if p_val <= 0.05:
                        ordreP = p
                        stat_data[family] = diff_log_series
                        break

stat_data = DataFrame(stat_data)
#print("\nRésultats finaux:")

df_diff = stat_data.copy()
#stat_data.dtypes

In [6]:
df_clust = read_csv("pf+clusters+dtw+complete.csv")
df_clust = df_clust.rename(columns={"clusters.norm": "clusters"}) 
df_clust = df_clust.drop(columns= "Unnamed: 0")

df_list_diffs = read_csv("stationnarity_steps.csv")

dictTransformations = df_list_diffs.set_index('family')

#df_clust

In [ ]:
maxlags_per_cluster = {
    1: 2,  # Cluster 1 avec maxlags = 4
    2: 4,  # Cluster 2 avec maxlags = 3
    3: 4,  # Cluster 3 avec maxlags = 2
}

for cluster in df_clust['clusters'].unique():
    # Sélectionner les familles de produits appartenant à ce cluster
    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    print("Cluster Numéro {0}".format(cluster))
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_diff[families_in_cluster].dropna()
    mlg = maxlags_per_cluster[cluster]
    # Exécuter le test de causalité de Granger pour le cluster
    granger_results = granger_causality_test(data = cluster_data, maxlag=mlg)
    
    # Afficher les résultats
    for (predictor, target), result in granger_results.items():
        p_values = [round(test[0]['ssr_ftest'][1], 4) for test in result.values()]
        print(f'Granger Causality test: {predictor} causes {target} with p-values {p_values}')


## division pour avoir le jeu d'entrainement et le jeu de test 

In [10]:
# Définir le point de division
split_point = int(len(df_diff) * 0.85)  # 85% des données pour l'entraînement

# Diviser les données
df_train = df_diff.iloc[:split_point]
df_test = df_diff.iloc[split_point:]

# Vérifier les tailles des ensembles d'entraînement et de test
print('Training set:', len(df_train))
print('Test set:', len(df_test))

Training set: 45
Test set: 8


### Choix du maxlag un peu au piff

## Choix du p

In [ ]:
maxlags_per_cluster = {
    1: 2,  # Cluster 1 avec maxlags = 4
    2: 4,  # Cluster 2 avec maxlags = 3
    3: 4,  # Cluster 3 avec maxlags = 2
}
orderVar = dict()
orderP = dict()
for cluster in df_clust['clusters'].unique():
    # Sélectionner les familles de produits appartenant à ce cluster
    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    print("Cluster Numéro {0}".format(cluster))
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
    
    # Ajuster le modèle VAR
    model = VAR(cluster_data)
    mlg = maxlags_per_cluster[cluster]
    # Choisir l'ordre optimal p pour le modèle VAR
    order_selection = model.select_order(maxlags=mlg)
    best_p = order_selection.aic  # Utilisez aic, bic, fpe ou hqic en fonction de votre préférence
    
    # Ajuster le modèle VAR avec le meilleur ordre p
    results = model.fit(best_p)
    print('order of var for the cluster {0} is {1}'.format(cluster,results.k_ar))
    orderVar[cluster] = results.k_ar
    orderP[cluster] = best_p
  

# Tests statistiques 

#### dictionnaire des ordre du modèle VAR par cluster
#### orderVar

## Affichage des résidus

In [ ]:
ordervarclust = { 2: 2,  3: 1, 1:1}
for cluster in df_clust['clusters'].unique():

    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
   
    print("Cluster Numéro {0}".format(cluster))
    model = VAR(cluster_data)
    order = ordervarclust[cluster]
    results = model.fit(order)
    residuals = results.resid
    for col in residuals.columns:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
        plot_acf(residuals[col], ax=axes[0])
        axes[0].set_title(f'Autocorrélation des résidus pour {col} du cluster {cluster}')
    
        plot_pacf(residuals[col], ax=axes[1])
        axes[1].set_title(f'Autocorrélation partielle des résidus pour {col} du cluster {cluster}')
    plt.show()

## les résidus du cluster 1 et 2 semblent être des bruits blancs

## Test de jarque_bera : test de normalité des erreurs 

## lhypothèse H0 est stipule que les données suivent une loi normale 
## si la p-value est < à 0.05, on rejète l'hypothèse nulle

In [ ]:
ordervarclust = { 2: 2,  3: 1, 1:1}
for cluster in df_clust['clusters'].unique():

    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
   
    print("Cluster Numéro {0}".format(cluster))
    model = VAR(cluster_data)
    order = ordervarclust[cluster]
    results = model.fit(order)
    residuals = results.resid
    for col in residuals.columns:
        print(f"Résidus pour {col}:")
        jb_test = jarque_bera(residuals[col])
        print(f"Test de Jarque-Bera: statistic={jb_test[0]}, p-value={jb_test[1]}")
        if jb_test[1] < 0.05:
            fig, ax = plt.subplots(figsize=(15, 5))
            # QQ-plot
            probplot(residuals[col], dist="norm", plot=ax)
            ax.set_title(f'QQ-plot {col} in cluster {cluster}')
            plt.show()  


## test de normalité multivaiée

In [24]:
ordervarclust = { 2: 2,  3: 1, 1:1}
for cluster in df_clust['clusters'].unique():

    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
   
    print("Cluster Numéro {0}".format(cluster))
    model = VAR(cluster_data)
    order = ordervarclust[cluster]
    results = model.fit(order)
    residuals = results.resid

    data = residuals
    print(multivariate_normality(data, alpha=.05) )



Cluster Numéro 2
HZResults(hz=0.9175576850625219, pval=0.11061067643715977, normal=True)
Cluster Numéro 3
HZResults(hz=0.7411591478580923, pval=0.22929233715876157, normal=True)
Cluster Numéro 1
HZResults(hz=168, pval=0.0, normal=False)


d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


## QQplot pour la normalité des résidus 

In [ ]:
ordervarclust = { 2: 2,  3: 1, 1:1}
qqFamily = "F_DTC"
for cluster in df_clust['clusters'].unique():

    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
   
    print("Cluster Numéro {0}".format(cluster))
    model = VAR(cluster_data)
    order = ordervarclust[cluster]
    results = model.fit(order)
    residuals = results.resid
    for col in residuals.columns:
        if col == qqFamily:
            fig, ax = plt.subplots(figsize=(15, 5))
            # QQ-plot
            probplot(residuals[col], dist="norm", plot=ax)
            ax.set_title(f'QQ-plot {col}')
            plt.show()

## le test ne semble pas passer pour la famille B du cluster 3

## Test de acorr_ljungbox : autocorrelation des erreurs 

## lhypothèse H0 stipule qu'il n'y a pas d'autocorrlation entre les erreurs d'ordre 1 à p
## si la p-value est < à 0.05, on rejète l'hypothèse nulle

In [14]:
ordervarclust = { 2: 2,  3: 1, 1:1}
maxlags_per_cluster = {
    1: 2,  # Cluster 1 avec maxlags = 4
    2: 4,  # Cluster 2 avec maxlags = 3
    3: 4,  # Cluster 3 avec maxlags = 2
}
for cluster in df_clust['clusters'].unique():

    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
    mlg = maxlags_per_cluster[cluster]
    print("Cluster Numéro {0}".format(cluster))
    model = VAR(cluster_data)
    order = ordervarclust[cluster]
    results = model.fit(order)
    residuals = results.resid
    results_list = list()
    #results_df = DataFrame(columns=['Variable', 'Lag', 'LB Statistic', 'LB P-value'])

    # Effectuer le test de Ljung-Box pour des décalages de 1 à maxlag
    for col in residuals.columns:
        for lag in range(1, mlg+1):
            lb_test = acorr_ljungbox(residuals[col], lags=[lag], return_df=True)
            results_list.append({
                'Variable': col,
                'Lag': lag,
                'LB Statistic': lb_test['lb_stat'].values[0],
                'LB P-value': lb_test['lb_pvalue'].values[0]
            })
    results_df = DataFrame(results_list)
    print(results_df)

Cluster Numéro 2
      Variable  Lag  LB Statistic  LB P-value
0        F_CRD    1      0.007047    0.933097
1        F_CRD    2      0.267037    0.875011
2        F_CRD    3      1.083437    0.781074
3        F_CRD    4      1.675076    0.795240
4        F_DEL    1      0.001390    0.970261
5        F_DEL    2      0.432202    0.805654
6        F_DEL    3      0.524954    0.913376
7        F_DEL    4      1.575323    0.813220
8   F_DIV_MCTP    1      0.339158    0.560316
9   F_DIV_MCTP    2      1.947289    0.377704
10  F_DIV_MCTP    3      2.418300    0.490237
11  F_DIV_MCTP    4      2.780548    0.595195
12       F_DPY    1      0.208223    0.648164
13       F_DPY    2      0.886944    0.641804
14       F_DPY    3      0.931914    0.817720
15       F_DPY    4      1.025156    0.905958
16       F_FIL    1      0.002970    0.956540
17       F_FIL    2      0.435820    0.804198
18       F_FIL    3      0.806713    0.847861
19       F_FIL    4      1.002452    0.909424
Cluster Numéro 3


d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
d:\WK\ydongue\AppData\Local\anaconda3\envs\devdaveyBickford\Lib\site-packages\statsmodels\tsa\stattools.py:693: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]


## auto correlation au niveau des lags 

In [ ]:
ordervarclust = { 2: 2,  3: 1, 1:1}
maxlags_per_cluster = {
    1: 2,  # Cluster 1 avec maxlags = 4
    2: 4,  # Cluster 2 avec maxlags = 3
    3: 4,  # Cluster 3 avec maxlags = 2
}
for cluster in df_clust['clusters'].unique():

    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
    mlg = maxlags_per_cluster[cluster]
    print("Cluster Numéro {0}".format(cluster))
    model = VAR(cluster_data)
    order = ordervarclust[cluster]
    results = model.fit(order)
    results.plot_acorr()

In [21]:
# 2sqrt(T)
print(2/sqrt(53))

0.27472112789737807


## Summary du modèle 

In [ ]:
ordervarclust = { 2: 2,  3: 1, 1:1}
maxlags_per_cluster = {
    1: 2,  # Cluster 1 avec maxlags = 4
    2: 4,  # Cluster 2 avec maxlags = 3
    3: 4,  # Cluster 3 avec maxlags = 2
}
for cluster in df_clust['clusters'].unique():

    families_in_cluster = df_clust[df_clust['clusters'] == cluster]['productfamilies']
    # Sélectionner les colonnes correspondantes dans le DataFrame des séries différentiées
    cluster_data = df_train[families_in_cluster].dropna()
    mlg = maxlags_per_cluster[cluster]
    print("Cluster Numéro {0}".format(cluster))
    model = VAR(cluster_data)
    order = ordervarclust[cluster]
    results = model.fit(order)
    print(results.summary())

## grande correlation entre E et F dans le cluster 2